# Demo Notebook for Hybrid CPW Algorithm with Anchors

In [ ]:
# Creates 3 transmon pockets in an L shape, each of which can be rotated in increments of 90 deg.
# Anchors are user-specified points through which the CPW must pass.
# For a specified step size and suitable choice of anchors, a snapped path can always be found.
# How close this path is to the shortest path depends on the step size - a smaller step size generally yields
# more optimal paths but requires a longer runtime.

In [ ]:
# TODO

# 1. CPW bounding boxes are not well-defined and cannot overlap right now. Bounding boxes should occupy as little
# area as possible to maximize space for other components.
    # Example: Connecting Q0_a and Q1_b ought to yield an S-shaped CPW but doesn't because there's supposedly a
    # bounding box with 0 area centered at the origin (0, 0).
# 2. Rebuilding distorts previously well-defined CPWs.
    # Example: Connect Q0_a, Q1_c, then connect Q1_b and Q0_d.
# 3. Enable anchor revision via the GUI.
# 4. Save a persistent state in the design so that rebuilding doesn't waste time.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings

design = designs.DesignPlanar()
gui = MetalGUI(design)

In [ ]:
design.overwrite_enabled = True

In [ ]:
from qiskit_metal.components.qubits.transmon_pocket import TransmonPocket

options = dict(
    pad_width = '425 um', 
    pocket_height = '650um',
    connection_pads=dict(  # pin connectors
        a = dict(loc_W=+1,loc_H=+1), 
        b = dict(loc_W=-1,loc_H=+1, pad_height='30um'),
        c = dict(loc_W=+1,loc_H=-1, pad_width='200um'),
        d = dict(loc_W=-1,loc_H=-1, pad_height='50um')
    )
)

q0 = TransmonPocket(design, 'Q0', options = dict(pos_x='-1.0mm', pos_y='-1.0mm', **options))
q1 = TransmonPocket(design, 'Q1', options = dict(pos_x='1.0mm', pos_y='+0.0mm', **options))
q2 = TransmonPocket(design, 'Q2', options = dict(pos_x='-1.0mm', pos_y='0.0mm', **options))

gui.rebuild()
gui.autoscale()

# 08/10/20: Can only build 1 CPW at a time due to aforementioned limitations!

# Therefore we delete old hybrid CPWs before building new ones.

In [ ]:
from qiskit_metal.components.interconnects.pathfinder import RoutePathfinder
ops=dict(fillet='90um')

In [ ]:
import numpy as np
from collections import OrderedDict

anchors = OrderedDict()
anchors[0] = np.array([0.048, -0.555])
anchors[1] = np.array([0.048, 0.195])

options = {'pin_inputs': 
            {'start_pin': {'component': 'Q0', 'pin': 'b'}, 
             'end_pin': {'component': 'Q1', 'pin': 'b'}},
            'lead': {'start_straight': '90um', 'end_straight': '90um'},
            'step_size': '0.25mm',
            'anchors': anchors,
            **ops
           }

qa = RoutePathfinder(design, 'line', options)

gui.rebuild()
gui.autoscale()

In [ ]:
design.delete_component('line')

gui.rebuild()

In [ ]:
anchors = OrderedDict()
anchors[0] = np.array([-0.452, -0.555])
anchors[1] = np.array([-0.452, -1.5])
anchors[2] = np.array([0.048, -1.5])

options = {'pin_inputs': 
            {'start_pin': {'component': 'Q0', 'pin': 'b'}, 
             'end_pin': {'component': 'Q1', 'pin': 'b'}},
            'lead': {'start_straight': '90um', 'end_straight': '90um'},
            'step_size': '0.25mm',
            'anchors': anchors,
            **ops
           }

qa = RoutePathfinder(design, 'line', options)

gui.rebuild()
gui.autoscale()

In [ ]:
design.delete_component('line')

gui.rebuild()

In [ ]:
options = {'pin_inputs': 
            {'start_pin': {'component': 'Q0', 'pin': 'a'}, 
             'end_pin': {'component': 'Q2', 'pin': 'd'}},
            'lead': {'start_straight': '90um', 'end_straight': '90um'},
            'step_size': '0.25mm',
            **ops
          }

qa = RoutePathfinder(design, 'line', options)

gui.rebuild()
gui.autoscale()

In [ ]:
design.delete_component('line')

gui.rebuild()

In [ ]:
anchors = OrderedDict()
anchors[0] = np.array([-2, 0.5])
anchors[1] = np.array([0, 0.5])
anchors[2] = np.array([0, -1])
anchors[3] = np.array([2, -1])

options = {'pin_inputs': 
            {'start_pin': {'component': 'Q0', 'pin': 'd'}, 
             'end_pin': {'component': 'Q1', 'pin': 'c'}},
            'step_size': '0.25mm',
            'anchors': anchors,
            **ops
           }

qc = RoutePathfinder(design, 'line', options)

gui.rebuild()
gui.autoscale()